In [ ]:
import tensorflow as tf
import numpy      as np
import pathlib
import data

from tensorflow.keras.optimizers  import Adam
from tensorflow.keras.losses      import CategoricalCrossentropy
from tensorflow.keras.layers      import Dense, Dropout, LSTM
from tensorflow.keras             import Sequential

In [ ]:
def getSequences(datasetType):
    sequences = []
    labels    = []
    for dataRow in dataObj.data[:]:
        if(datasetType == dataRow[0]):
            sequencePath = pathlib.Path(r"D:\ActionRecognition\Sequences")/dataRow[0]/dataRow[1]/(dataRow[2] + "_featureSequence.npy")
            sequence     = np.load(sequencePath)
            sequences.append(sequence)
            label = dataObj.getClassIndex(dataRow[1])
            labels.append(label)
    return np.array(sequences), np.array(labels, dtype=np.uint8)

In [ ]:
def getDataset(sequences, labels):
    dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
    return dataset

In [ ]:
def prepareTrainDataset(dataset, cache, shuffleBufferSize):
    if cache:
        if isinstance(cache, str):
            dataset = dataset.cache(cache)
        else:
            dataset = dataset.cache()
    
    dataset = dataset.shuffle(buffer_size = shuffleBufferSize)
    # Repeat forever
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    # `prefetch` lets the dataset fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [ ]:
def getModel():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(256, input_shape=(None, 2048)))
    model.add(tf.keras.layers.Dense(64, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(numClasses, activation = 'softmax')) # change 3 to numClasses
    
    model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.0001),
                  loss      = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
                  metrics   = ['accuracy'])
    return model

In [ ]:
def trainModel(model, epochs, trainDataset, validationDataset, steps_per_epoch, validation_steps, callbacks):
    history = model.fit(trainDataset, 
                        epochs = epochs,
                        validation_data  = validationDataset,
                        steps_per_epoch  = steps_per_epoch,
                        validation_steps = validation_steps,
                        callbacks = callbacks)
    return model, history

In [ ]:
dataObj    = data.Data()
numClasses = dataObj.numClasses
AUTOTUNE   = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

In [ ]:
def main():
    epochs = 10
    
    cacheFilePath = "./trainRNNDatasetCache"
    
    trainSequences, trainLabels           = getSequences('Train')
    validationSequences, validationLabels = getSequences('Validation')
    
    trainDataset      = getDataset(trainSequences, trainLabels)
    validationDataset = getDataset(validationSequences, validationLabels)
    
    trainDataset      = prepareTrainDataset(trainDataset, cacheFilePath, 100)
    validationDataset = validationDataset.batch(BATCH_SIZE)
    
    trainSeqCount      = dataObj.getDatasetCount('Train')
    validationSeqCount = dataObj.getDatasetCount('Validation')
    
    steps_per_epoch  = np.ceil(trainSeqCount/BATCH_SIZE)
    validation_steps = np.ceil(validationSeqCount/BATCH_SIZE)
    
    model = getModel()
    trained_model, history = trainModel(model, epochs, trainDataset, validationDataset, steps_per_epoch, validation_steps, [])

In [ ]:
main()